In [ ]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta


In [ ]:
product_type = "frozen"
num_time_periods = 7

In [ ]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [ ]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [ ]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [ ]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [ ]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu,minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [ ]:
def createDemand(file, articles):
    forecast_data = pd.read_csv(file)
    dates_list = [datetime.strptime(date, '%Y-%m-%d').date() for date in forecast_data['DATE'].unique()]
    dates = [min(dates_list)+timedelta(days=x) for x in range((max(dates_list)-min(dates_list)).days + 1)]
    time_periods = [date.strftime("%Y-%m-%d") for date in dates]

    forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]
    demand = dict.fromkeys(articles)
    for item in demand.keys():
        demand[item] = dict.fromkeys(time_periods)
        for time_period in demand[item].keys():
            count = forecast_data.loc[(forecast_data['ARTICLE_ID'] == item) & (forecast_data['DATE'] == time_period)]['PICKING_QUANTITY_CU']
            if(len(count)==0):
                demand[item][time_period] = 0
            else:
                demand[item][time_period] = count.values[0]
    return demand

In [ ]:
def fetchDemand(num_time_periods, product_type):
    with open("demand_"+product_type+"_"+str(num_time_periods)+".pkl","rb") as f:
        return pkl.load(f)

In [ ]:
# demand = createDemand('./data/sales_7.csv', articles)
demand = fetchDemand(num_time_periods, product_type)

In [ ]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_periods, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_periods, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, time_period] for item in items for time_period in time_periods)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[time_period] for time_period in time_periods)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in items:
    for time_period_iterator, time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr(orders[item, time_period] * cu_per_tu[item] >= demand[item][time_period],name="demand constraint_" + str(time_period))
        else:
            m.addConstr((orders[item, time_period] * cu_per_tu[item]) + storage_used[item, time_periods[time_period_iterator-1]] >= demand[item][time_period],name="demand constraint_" + str(time_period))

# inventory volume constraint
for time_period_iterator, time_period in enumerate(time_periods):
    if(time_period_iterator==0):
        m.addConstr(gp.quicksum(volume_per_cu[item] * ((cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])
    else:
        m.addConstr(gp.quicksum(volume_per_cu[item] * (storage_used[item, time_periods[time_period_iterator-1]] + (cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])

# # min/max constraints (linking too)
for item in items:
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= minimum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # if((not math.isnan(maximum_order_quantity_tu[item]))):
        #     m.addConstr(orders[item, time_period] <= maximum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # else:
        #     m.addConstr(orders[item, time_period] <= default_max_order * ordered_boolean[item, time_period])

# current inventory constraint
for item in items:
    for time_period_iterator,time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr(storage_used[item, time_periods[time_period_iterator]] == (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])
        else:
            m.addConstr(storage_used[item, time_periods[time_period_iterator]] == storage_used[item, time_periods[time_period_iterator-1]] + (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])

m.setParam(gp.GRB.Param.Heuristics, 0) 
m.optimize()